# MBTA Part 3
This notebook to to test code to get object working, write code to estimate non-payroll costs

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import Agency_Classes

In [21]:
app_token = "2Qa1WiG8G4kj1vGVd2noK7zP0"
client = Socrata("cthru.data.socrata.com", app_token)
yr = list(range(2016,2021))

In [31]:
MBTA = Agency_Classes.MBTA(alias="MBT", official_name="MASSACHUSETTS BAY TRANSPORTATION AUTHORITY (MBT)",
                                    year_range=[x for x in yr if x > 2016],  # MBT only goes back to 2017
                               payroll_vendors=["MBT - SUMMARY PAYROLL"],
                               payroll_official_name="Massachusetts Bay Transportation Authority (MBT)", client=client,
                                    category="Police")

In [32]:
MBTA.get_payroll_by_year()

In [33]:
MBTA.payroll_by_year[[2016, 2020]] = np.NaN
MBTA.payroll_by_year

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

Ok, that looks good. I was going to look at % of total expenditures that state police spend on payroll, use that to caculate expenditures for MBTA. But I'll hold off on that will ask Bobby if I get a chance but better to work on other stuff for now. Also think I will use assumption that 50% of MBTA police arrests go to suffolk county for population correction I bet that's an undercount but is ok for now

Get 2015 payroll numbers, use to approximate 2016 numbers

In [4]:
import MBTA_Payroll_Scraper

In [5]:
MBTA_payroll_dict_2015 = MBTA_Payroll_Scraper.scrape_payroll()

got to page 0
got total pay of  169110.73
got cop pay of 2941.5
got to page 1
got total pay of  646849.81
got cop pay of 2941.5
got to page 2
got total pay of  1420322.4099999997
got cop pay of 2941.5
got to page 3
got total pay of  2433896.6899999995
got cop pay of 36850.98
got to page 4
got total pay of  3735189.8099999996
got cop pay of 105042.13
got to page 5
got total pay of  5403172.2299999995
got cop pay of 130503.93000000001
got to page 6
got total pay of  7413765.71
got cop pay of 130503.93000000001
got to page 7
got total pay of  9743154.37
got cop pay of 130503.93000000001
got to page 8
got total pay of  12293257.959999997
got cop pay of 166201.6
got to page 9
got total pay of  14815185.819999997
got cop pay of 166201.6
got to page 10
got total pay of  17609224.849999998
got cop pay of 166201.6
got to page 11
got total pay of  20512972.849999998
got cop pay of 166201.6
got to page 12
got total pay of  23546077.069999997
got cop pay of 166201.6
got to page 13
got total pay of

In [6]:
MBTA_payroll_dict_2015

{2015: {'total_pay_actual': 539586661.4599999,
  'police_pay': 28140924.850000005},
 2017: {'total_pay_actual': 0, 'police_pay': 0}}

In [15]:
from PyPDF2 import PdfFileReader
from MBTA_Payroll_Scraper import page_to_image
import pdf2image
import pytesseract

In [16]:
mbta_pdfObj = PdfFileReader("data/MBTA_pdfs/mbta-payroll-2015.pdf")
temp_path = "data/MBTA_pdfs/mbta_temp_page.pdf"

In [17]:
page_to_image(mbta_pdfObj, 0, temp_path)
images_from_path = pdf2image.convert_from_path(temp_path)
row_list = pytesseract.image_to_string(images_from_path[0], config='--psm 6').split("\n")

In [18]:
row_list

['Kenneth Murphy Rail Maint OrangeWellington Cleaner,Car 433 61776.000 0.00',
 'Leonard McMahon Bus Transportation ChritwnFels Operator,Surface 126 40955.200 0.00',
 'Omar Trochez Bus Transportation Arborway Operator,Surface 122 72779.200 0.00',
 'Mark McLaughlin TFM Bldg & Station Maintenance Supv Night Cleaning 309 73296.340 0.00',
 'Jose Agosto Police Officer,Police 741 0.000 0.00',
 'Barbara Dixon EVE Everett Main Repair -Bus Driver, Truck a4 72508.800 0.00',
 'Debra Spinzola C_MKT Customer Communications Rep,Customer Support 92 56030.780 0.00',
 'Daniel Preston Bus Transportation Bennett Operator,Surface 125 72779.200 0.00',
 'Deborah McKenna HRail Transp Red JFK/UMASS Motorperson,Rapid Transit 113 75691.200 0.00',
 'Veronica Roberts HRail Transp Red JFK/UMASS Motorperson,P-T 113 56768.400 0.00',
 'Derrick Napier HRail Transp Blue Orient Hght Motorperson,P-T RTL Yard 114 54584.400 0.00',
 'Maraya Sierra Bus Transportation CabotAlbany Operator,P-T Surface 123 40934.400 0.00',
 'Leo

In [23]:
def get_pay(row_list, split_on):
    total_pay = 0
    total_cop_pay = 0
    for row in row_list:
        try:
            pay = float(row.split(split_on)[-1].replace(",",""))
            total_pay += pay
            if "Police" in row:
                    total_cop_pay += pay
        except:
            pass
    return total_pay, total_cop_pay

In [24]:
get_pay(row_list, " ")

(169110.73, 2941.5)